<a href="https://colab.research.google.com/github/confifu/TryOnGAN-pytorch/blob/main/notebooks/PC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
%%capture
!pip install wandb==0.9.7

In [ ]:
import wandb
#wandb.login()

from google.colab import drive
drive.mount('/gdrive')


Mounted at /gdrive


In [ ]:
#clone one

!git clone -b PC-concat https://github.com/confifu/TryOnGAN-pytorch.git
#!git clone -b PC-add https://github.com/confifu/TryOnGAN-pytorch.git

# Generate Dataset(Images)



In [ ]:
%%capture
%cd /content/
!gdown https://drive.google.com/uc?id=1r_Pz2gk1Sp2sjYS7cj9oYUMbLGo9EyZS
!unzip /content/inshopclothes.zip
!rm /content/inshopclothes.zip

In [ ]:

%cd /content/TryOnGAN-pytorch

!python dataset_tool.py --source=/content/train --dest=/content/my_dataset --width=256 --height=256
!rm -rf /content/train

# Generate pose keypoints

In [ ]:
%%capture
'''
!mkdir /content/new_images
!unzip /content/my_dataset.zip -d "/content/new_images/"
!mkdir /content/flat_images
!find /content/new_images -mindepth 2 -type f -exec mv -t /content/flat_images -i '{}' +
!rm -rf /content/new_images
!mv /content/flat_images /content/new_images
'''

In [ ]:
'''
!unzip /content/TestImages.zip
!mv /content/TestImages /content/new_images
'''

In [ ]:
#create pose keypoints
'''

! pip install pyyaml==5.2
! pip install scipy==1.1.0
! pip install torch==1.2.0 torchvision==0.4.0
! pip install pillow==6.2.2
!python -m pip install cython
!sudo apt-get install libyaml-dev

import yaml, scipy, os, torch
import pandas as pd


os.chdir('/content/')
!rm -rf /content/AlphaPose
!git clone https://github.com/MVIG-SJTU/AlphaPose.git

os.chdir('/content/AlphaPose')
! python setup.py build develop

!gdown https://drive.google.com/uc?id=1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC
!gdown https://drive.google.com/uc?id=1nlnuYfGNuHWZztQHXwVZSL_FvfE551pA
!gdown https://drive.google.com/uc?id=1kQhnMRURFiy7NsdS8EFL-8vtqEXOgECn

!mkdir /content/AlphaPose/detector/yolo/data
!mkdir /content/AlphaPose/detector/tracker/data

!mv /content/AlphaPose/yolov3-spp.weights /content/AlphaPose/detector/yolo/data/yolov3-spp.weights
!mv /content/AlphaPose/jde.1088x608.uncertainty.pt /content/AlphaPose/detector/tracker/data/JDE-1088x608-uncertainty
!mv /content/AlphaPose/fast_res50_256x192.pth /content/AlphaPose/pretrained_models/fast_res50_256x192.pth

!(python3 scripts/demo_inference.py \
--cfg configs/coco/resnet/256x192_res50_lr1e-3_1x.yaml \
--checkpoint pretrained_models/fast_res50_256x192.pth \
--indir /content/new_images/ \
--sp \
--qsize 128)
# result json and rendered images are saved here:


! ls examples/res/

import json
def createPoseAnnotation(jsonFileName):
    
    imageSize = 256.0
    heatMapSize = 64.0
    ratio = heatMapSize / imageSize
    jsonFile = open(jsonFileName)
    data = json.load(jsonFile)
    print("Number of pose keypoints generated ", len(data))

    files = []
    keypoints = []
    for i in range(len(data)):
        name = data[i]['image_id']
        files.append(name)

        keypoint = data[i]['keypoints']
        string = ':'.join(str(e) for e in keypoint)
        keypoints.append(string)

    
    data = {'name' : files,
            'keypoints' : keypoints}

    newdf = pd.DataFrame.from_dict(data)
    newdf.to_csv('/content/pose-annotations.csv', index = False)

    jsonFile.close()

createPoseAnnotation('/content/AlphaPose/examples/res/alphapose-results.json')
'''


# Download pose keypoints from gdrive



In [ ]:
%cd /content
!gdown https://drive.google.com/uc?id=1XryABeNxvzsdjEY7j4ed2m4G-we0n0md

# Train model

In [ ]:

%cd /content/TryOnGAN-pytorch

!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force

In [ ]:
!pip install torch==1.8.1 torchvision==0.9.1

In [ ]:
#for training ffhq checkpoint model
%cd /content/TryOnGAN-pytorch
!(python train.py \
--outdir=/content/outdir \
--data=/content/my_dataset.zip \
--posefile=/content/pose-annotations.csv \
--resume=/gdrive/MyDrive/ada-tryongan/best_checkpoints/PC-add-scratch-network-snapshot-002040.pkl \
--mirror=1 \
--gpus=1 \
--seed=600 \
--batch=32 \
--aug=ada \
--snap=1 \
--project=ada-uncon-scratch)

function ClickConnect(){
    console.log("Clicked on connect button"); 
    document.querySelector("colab-connect-button").click()
}
setInterval(ClickConnect,60000)